In [1]:
import sys
import redcap
from PyQt5.QtWidgets import QDesktopWidget,QMainWindow, QApplication, QWidget, QPushButton, QAction, QLineEdit, QMessageBox,QFileDialog,QLabel,QApplication,QComboBox,QPlainTextEdit,QSplitter,QGroupBox, QButtonGroup, QRadioButton
from PyQt5.QtGui import QIcon, QFont, QPixmap, QImage

from PyQt5.QtCore import *
import pandas as pd
import numpy as np
import cv2
import pyzbar.pyzbar as pyzbar
from pyzbar.wrapper import ZBarSymbol
from playsound import playsound
import os
from datetime import datetime
import shutil

from barcodeGenerator import BarcodeGenerator
from labelPrintHelper_mac import LabeweblPrintHelper_mac
from redcapHelper import RedcapHelper
from barcode_reader import Barcode_reader

class RedcapLabelwriterApp_v3(QMainWindow):

    def __init__(self):
        super().__init__()
        self.title = 'GCA Redcap LabelPrinter Toolkit_v1'
        self.left = 10
        self.top = 10
        self.width = 800 # 400
        self.height = 585 # 585
        self.initUI()
        # create redcap instance
        f_redcap_key = open("REDCAP_API_KEY.txt", "r")
        self.redcapExec = RedcapHelper(str(f_redcap_key.read()))
        
        # class vairables definition for getting value from GUI
        self.subjectId = ''
        self.studyType = ''
        self.processed_by = ''
        self.action_comment = ''
        self.barcode_id = ''
        self.freezer = ''
        self.rack = ''
        self.box_id = ''
        self.box_position = ''
        self.stats_type = ''
        
        self.th_webcam = Barcode_reader()
        self.th_webcam.change_pixmap.connect(self.set_webcam_image)
        self.th_webcam.found_qr.connect(self.set_scanned_id)
        
        curDir = os.getcwd()
        barcode_tmp_path = curDir + '/tmp'
        try:
            os.mkdir(barcode_tmp_path)
        except OSError:
            print ("Creation of the directory %s failed" % barcode_tmp_path)
        
        # setup default label printer use system command. 
        os.system('lpoptions -d DYMO_LabelWriter_450')

    def initUI(self):
        """
        UI vairable setup
        """
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        self.centerOnScreen()
        
        myFont=QFont()
        myFont.setBold(True)
       
        self.label_setup = QLabel(self)
        self.label_setup.resize(360, 20)
        self.label_setup.move(20, 15)
        self.label_setup.setText('--------Subject Setup------------------------------------')
        self.label_setup.setFont(myFont)        
        
        self.label_subject_id = QLabel(self)
        self.label_subject_id.resize(130, 20)
        self.label_subject_id.move(20, 40)
        self.label_subject_id.setText('Subject ID')
        
        
        self.textbox_subject_id = QLineEdit(self)
        self.textbox_subject_id.resize(130,20)
        self.textbox_subject_id.move(150, 40)
        
        self.btn_get_next_subject = QPushButton('Next Subject', self)
        self.btn_get_next_subject.resize(120,30)
        self.btn_get_next_subject.move(280,35)
        
        
        self.label_study_type = QLabel(self)
        self.label_study_type.resize(130, 20)
        self.label_study_type.move(20, 65)
        self.label_study_type.setText('Study Type')
        

        self.cb_study_type = QComboBox(self)
        self.cb_study_type.resize(130,20)
        self.cb_study_type.move(150,65)
        self.cb_study_type.addItem('')
        self.cb_study_type.addItems(["CD_ENDO", "CTL_ENDO", "CD_Surgery","CTL_Surgery"])    
        
        self.btn_get_subject_type = QPushButton('Subject Type', self)
        self.btn_get_subject_type.resize(120,30)
        self.btn_get_subject_type.move(280,60)
        
        self.label_processed_by = QLabel(self)
        self.label_processed_by.resize(130, 20)
        self.label_processed_by.move(20, 90)
        self.label_processed_by.setText('Custom Option:')
        
        self.cb_option_type = QComboBox(self)
        self.cb_option_type.resize(230,20)
        self.cb_option_type.move(150,90)
        self.cb_option_type.addItem('')
        self.cb_option_type.addItems(["Default",
                                      "Need extra FROZEN specimens", 
                                     "No FRESH specimens, NO extra FROZEN specimens", 
                                     "No FRESH specimens, ADD extra FROZEN specimens"])
        
#         self.radio_yes = QRadioButton(self)
#         self.radio_yes.setChecked(False)
#         self.radio_yes.resize(50, 20)
#         self.radio_yes.move(450, 90)
#         self.radio_yes.setText("yes")
       
        
#         self.radio_no = QRadioButton(self)
#         self.radio_no.setChecked(False)
#         self.radio_no.resize(50, 20)
#         self.radio_no.move(600, 90)
#         self.radio_no.setText("no")

        
#         self.label_processed_by = QLabel(self)
#         self.label_processed_by.resize(200, 20)
#         self.label_processed_by.move(20, 90)
#         self.label_processed_by.setText('Extra Frozen Specimen?')
        
#         self.radio_yes = QRadioButton(self)
#         self.radio_yes.setChecked(False)
#         self.radio_yes.resize(50, 20)
#         self.radio_yes.move(150, 90)
#         self.radio_yes.setText("yes")
       
        
#         self.radio_no = QRadioButton(self)
#         self.radio_no.setChecked(False)
#         self.radio_no.resize(50, 20)
#         self.radio_no.move(200, 90)
#         self.radio_no.setText("no")
        
        
        self.label_processed_by = QLabel(self)
        self.label_processed_by.resize(130, 20)
        self.label_processed_by.move(20, 115)
        self.label_processed_by.setText('Processed by')
             
        self.cb_processed_by = QComboBox(self)
        self.cb_processed_by.resize(130,20)
        self.cb_processed_by.move(150, 115)
        self.cb_processed_by.addItem('')
        self.cb_processed_by.addItems(["Sophie", "Regina"])  
        
        self.btn_get_last_printed = QPushButton('Last Printed', self)
        self.btn_get_last_printed.resize(120,30)
        self.btn_get_last_printed.move(280,110)
                
        self.label_comment = QLabel(self)
        self.label_comment.resize(130, 20)
        self.label_comment.move(20, 135)
        self.label_comment.setText('Comment / Note')
        
        self.plain_comment = QPlainTextEdit(self)
        self.plain_comment.resize(240,80)
        self.plain_comment.move(150, 135)
        
        self.masi_label = QLabel(self)
        self.masi_label.resize(82, 65)
        self.masi_label.move(20, 155)
        pixmap = QPixmap('masi.png')
        self.masi_label.setPixmap(pixmap.scaled(82,65,Qt.KeepAspectRatio))
        

                
        self.label_new_pack = QLabel(self)
        self.label_new_pack.resize(360, 20)
        self.label_new_pack.move(20, 220)
        self.label_new_pack.setText('--------Print Barcode------------------------------------')
        self.label_new_pack.setFont(myFont)
        
#         self.btn_gen_new_pack = QPushButton('Generate and Print\nnew Barcode Pack', self)
        self.btn_gen_new_pack = QPushButton('Full Pack', self)
        self.btn_gen_new_pack.resize(120,30)
        self.btn_gen_new_pack.move(20,245)
        
        self.btn_print_frozen_only = QPushButton('Frozen only', self)
        self.btn_print_frozen_only.resize(120,30)
        self.btn_print_frozen_only.move(140,245)
        
        self.btn_print_serum_only = QPushButton('Serum only', self)
        self.btn_print_serum_only.resize(120,30)
        self.btn_print_serum_only.move(260,245)
        
        self.label_single_barcode = QLabel(self)
        self.label_single_barcode.resize(360, 20)
        self.label_single_barcode.move(20, 280)
        self.label_single_barcode.setText('--------Single Barcode Action------------------------------------')
        self.label_single_barcode.setFont(myFont)
        
        self.label_barcode_id = QLabel(self)
        self.label_barcode_id.resize(130, 20)
        self.label_barcode_id.move(20, 305)
        self.label_barcode_id.setText('Barcode ID')
        
        # Create textbox
        self.textbox_barcode_id = QLineEdit(self)
        self.textbox_barcode_id.resize(170,20)
        self.textbox_barcode_id.move(105, 305)
        
        self.btn_scanned = QPushButton('Webcam', self)
        self.btn_scanned.resize(120,25)
        self.btn_scanned.move(280,300)     
        w
        self.label_action_type = QLabel(self)
        self.label_action_type.resize(80, 30)
        self.label_action_type.move(20, 330)
        self.label_action_type.setText('Action List')

        self.cb_action_type = QComboBox(self)
        self.cb_action_type.resize(230,30)
        self.cb_action_type.move(100,330)
        self.cb_action_type.addItem('')
        self.cb_action_type.addItems(['Re-print',
                                      'DNA distributed to Vantage', 
                                      'DNA extracted and banked', 
                                      'Distributed to Lau',
                                      'Distributed to TPSR',
                                      'Paraffin blocks back from TPSR',
                                      'Distributed to Vantage'])   
        
        
        self.btn_sent_to = QPushButton('Run', self)
        self.btn_sent_to.resize(70,30)
        self.btn_sent_to.move(330,330)

        self.cam_label = QLabel(self)
        self.cam_label.resize(340, 200)
        self.cam_label.move(35, 365)
        
        pixmap = QPixmap('GCA.png')
        self.cam_label.setPixmap(pixmap)
        
        self.label_single_barcode = QLabel(self)
        self.label_single_barcode.resize(360, 20)
        self.label_single_barcode.move(420, 15)
        self.label_single_barcode.setText('--------Stats Summary------------------------------------')
        self.label_single_barcode.setFont(myFont)
        
        self.label_stats_type = QLabel(self)
        self.label_stats_type.resize(90, 20)
        self.label_stats_type.move(420, 50)
        self.label_stats_type.setText('Stats Type')
        
        self.cb_stats_type = QComboBox(self)
        self.cb_stats_type.resize(90,20)
        self.cb_stats_type.move(510,50)
        self.cb_stats_type.addItem('')
        self.cb_stats_type.addItems(["Project", "Subject", "Barcode"])
        
        self.label_stats_type = QLabel(self)
        self.label_stats_type.resize(90, 20)
        self.label_stats_type.move(710, 45)
        self.label_stats_type.setText('Version 1.0.0')
        
        self.label_contact_us = QLabel(self)
        self.label_contact_us.resize(90, 40)
        self.label_contact_us.move(710, 60)
        self.label_contact_us.setText(" <a href=\"mailto:shunxing.bao@vanderbilt.edu\">Contact us</a>")
        self.label_contact_us.setOpenExternalLinks(True)
        
        self.btn_stats = QPushButton('Print Stats', self)
        self.btn_stats.resize(180,30)
        self.btn_stats.move(420,70)

        self.plain_console = QPlainTextEdit(self)
        self.plain_console.resize(360,465)
        self.plain_console.move(420, 100)
        self.plain_console.setPlainText('--------------Output Log--------------')

        # connect button to on_click functions
        self.btn_get_next_subject.clicked.connect(self.getNextSubject)
        self.btn_gen_new_pack.clicked.connect(self.genPrintNewPack)
        self.btn_get_subject_type.clicked.connect(self.getSubjectType)
        self.btn_get_last_printed.clicked.connect(self.getLastPrinted)
        self.btn_sent_to.clicked.connect(self.setStatusSetAction)
        self.btn_scanned.clicked.connect(self.setStatusScanned)        
        self.btn_stats.clicked.connect(self.getBarcodeStats)
        self.btn_print_frozen_only.clicked.connect(self.printExtraFrozenOnly)
        self.btn_print_serum_only.clicked.connect(self.printExtraSerumOnly)
        
####         self.radio_yes.clicked.connect(self.ifNeedExtraFrozen)
####         self.radio_no.clicked.connect(self.ifNeedExtraFrozen)
        # return enter event
        self.textbox_barcode_id.returnPressed.connect(self.getBarcodeFromScanner)
        # show the UI
        self.show()
    
        
    def centerOnScreen (self):
        '''centerOnScreen()
Centers the window on the screen.'''
        resolution = QDesktopWidget().screenGeometry()
        self.move((resolution.width() / 2) - (self.frameSize().width() / 2),
                  (resolution.height() / 2) - (self.frameSize().height() / 2)) 
        
    def getGUIFieldValue(self):
        """
        Get all filed values from UI for laziness
        """
        self.subjectId = self.textbox_subject_id.text()
            
        self.studyType = self.cb_study_type.currentText()
        self.processed_by = self.cb_processed_by.currentText()
        self.custom_option = self.cb_option_type.currentText()
        self.action_comment = self.plain_comment.toPlainText()
        self.barcode_id = self.textbox_barcode_id.text()
        self.action_type = self.cb_action_type.currentText()
        self.stats_type = self.cb_stats_type.currentText()
        
    
#     @pyqtSlot()
#     def ifNeedExtraFrozen(self):
#         """
#         Simple radio button yes or no selection
#         """
#         if self.radio_yes.isChecked():
#             print('nima')
#             self.radio_no.setChecked(False)
#             return
#         if self.radio_no.isChecked():
#             print('nima2')
#             self.radio_yes.setChecked(False)
#             return
        
    @pyqtSlot()
    def getNextSubject(self):
        """
        The idea is to get next not printed subject id with sequential order. 
        Probably not very useful right now because patient may not show up in order
        """
        barcodeSubset =  self.redcapExec.getBarcodeSubset()
        next_subject_id = ''
  
        # if nothing printed
        if len(barcodeSubset) == 0 :
            next_subject_id = 1
        else:
            df = pd.DataFrame.from_dict(barcodeSubset)
            barcodeSubsetList  = df['barcode_sample_id'].values


            printBarcodeSubsetList = []
            for x in barcodeSubsetList:
                printBarcodeSubsetList.append(int(x[3:6]))

            printBarcodeSubsetList_np = np.asarray(printBarcodeSubsetList)
            printBarcodeSubsetList_np_unique = np.unique(printBarcodeSubsetList_np)
            
            if len(printBarcodeSubsetList_np_unique) is 0:
                next_subject_id = 1
            else:
                tmp = 0
                for i in printBarcodeSubsetList_np_unique:
                    if i - tmp == 1:
                        tmp = i
                        next_subject_id = i +  1
                    else:
                        next_subject_id = tmp +  1
                        break

        # Query from Sophie's redcap form to get existent patient entries       
        curSubj = self.redcapExec.getInputPatient()
        
        df = pd.DataFrame.from_dict(curSubj)
        record = df.loc[df['record_id_dem_endo'] == str(next_subject_id)]

        if len(record) is 0 or None:
            QMessageBox.about(self, "WARNING", '- Please create subject %s form first' % str(next_subject_id))
            return
            
        next_print_subject_id_verified = self.getPatentId(next_subject_id)
        self.textbox_subject_id.setText(next_print_subject_id_verified)
        
        studyType_redcap = record.get('redcap_event_name').values
        
        #"CD_ENDO", "CTL_ENDO", "CD_Surgery","CTL_Surgery"
        if studyType_redcap == 'cd_arm_1':
            self.cb_study_type.setCurrentIndex(1)
        elif studyType_redcap == 'control_arm_1':
            self.cb_study_type.setCurrentIndex(2)
        elif studyType_redcap == 'cd_arm_2':
            self.cb_study_type.setCurrentIndex(3)
        elif studyType_redcap == 'control_arm_2':
            self.cb_study_type.setCurrentIndex(4)
        
        self.cb_study_type.repaint()
    
#     @pyqtSlot()
#     def getCustomOption(self):
#         """
#         Get selected subject id
#         """
#         self.getGUIFieldValue()
#         if self.subjectId == '' or self.subjectId == None:
#             QMessageBox.about(self, "WARNING", "Subject id is empty. Get subject %s type failed." % str(self.subjectId)  )
#             return
        
#         curSubj = self.redcapExec.getInputPatient()
        
#         df = pd.DataFrame.from_dict(curSubj)
#         record = df.loc[df['record_id_dem_endo'] == str(int(self.subjectId))]

#         if len(record) is 0 or None:
#             QMessageBox.about(self, "WARNING", '-- Please create subject %s form first' % str(int(self.subjectId)))
#             return
        
#         studyType_redcap = record.get('redcap_event_name').values
        
#         #"CD_ENDO", "CTL_ENDO", "CD_Surgery","CTL_Surgery"
#         if studyType_redcap == 'cd_arm_1':
#             self.cb_study_type.setCurrentIndex(1)
#         elif studyType_redcap == 'control_arm_1':
#             self.cb_study_type.setCurrentIndex(2)
#         elif studyType_redcap == 'cd_arm_2':
#             self.cb_study_type.setCurrentIndex(3)
#         elif studyType_redcap == 'control_arm_2':
#             self.cb_study_type.setCurrentIndex(4)
        
#         self.cb_study_type.repaint()
        
    def getSubjectType(self):
        """
        Get selected subject id
        """
        self.getGUIFieldValue()
        if self.subjectId == '' or self.subjectId == None:
            QMessageBox.about(self, "WARNING", "Subject id is empty. Get subject %s type failed." % str(self.subjectId)  )
            return
        
        curSubj = self.redcapExec.getInputPatient()
        
        df = pd.DataFrame.from_dict(curSubj)
        record = df.loc[df['record_id_dem_endo'] == str(int(self.subjectId))]

        if len(record) is 0 or None:
            QMessageBox.about(self, "WARNING", '-- Please create subject %s form first' % str(int(self.subjectId)))
            return
        
        studyType_redcap = record.get('redcap_event_name').values
        
        #"CD_ENDO", "CTL_ENDO", "CD_Surgery","CTL_Surgery"
        if studyType_redcap == 'cd_arm_1':
            self.cb_study_type.setCurrentIndex(1)
        elif studyType_redcap == 'control_arm_1':
            self.cb_study_type.setCurrentIndex(2)
        elif studyType_redcap == 'cd_arm_2':
            self.cb_study_type.setCurrentIndex(3)
        elif studyType_redcap == 'control_arm_2':
            self.cb_study_type.setCurrentIndex(4)
        
        self.cb_study_type.repaint()
    
    @pyqtSlot()
    def getLastPrinted(self):
        """
        Get last printed subject id
        """
        tmp_subset = self.redcapExec.project.export_records(events=['action_tuple_table_arm_3'],fields=['barcode_sample_id','barcode_action_type','barcode_processed_by'])
        barcodeSubset = self.redcapExec.getBarcodeSubset()
        if len(barcodeSubset) == 0: 
            QMessageBox.about(self, "WARNING", 'No subject got printed yet' )
            return

        df = pd.DataFrame.from_dict(tmp_subset)
        record = df.loc[df['barcode_action_type'] == 'printed']

        if len(record) is 0 or None:
            QMessageBox.about(self, "WARNING", 'Something wrong, please contact administrator!')
        else:
#            record_list = record['barcode_sample_id'].tolist()
#            last_print_subj = record_list[len(record_list)-1][3:6]
            
            last_record = df.iloc[-1,:]
            last_record_list = last_record.tolist()
            last_print_subj = last_record_list[2][3:6] # index 2 is 'barcode_sample_id'
            last_processed_by = last_record_list[3] # the last element
                                                    #'action_tuple_table_arm_3',
                                                    #'barcode_sample_id',
                                                    #'barcode_action_type',
                                                    #'barcode_processed_by'
            QMessageBox.about(self, "WARNING", 'Last printed subject: %s\nProcessed by: %s' % (str(last_print_subj),str(last_processed_by)))
    
    @pyqtSlot()
    def printExtraFrozenOnly(self):
        """
        Generate and print new pack of barcodes
        """
        self.getGUIFieldValue()
        if self.subjectId == '' or self.subjectId == None:
            QMessageBox.about(self, "WARNING", "Subject id is empty. Print extra frozen specimen for subject %s failed." % str(self.subjectId)  )
            return
        
        if len(self.subjectId) != 3:
            QMessageBox.about(self, "WARNING", "Subject id should be three digit. Please double-check" )
            return
        
        if self.studyType == '' or self.studyType == None:
            QMessageBox.about(self, "WARNING", "Study type is empty. Generate / Print new pack for subject %s failed." % str(self.subjectId)  )
            return
        
        if self.processed_by == '' or self.processed_by ==None:
            QMessageBox.about(self, "WARNING", "Please select current user." )
            return
        

        
        curSubj = self.redcapExec.getInputPatient()
        df = pd.DataFrame.from_dict(curSubj)
        record = df.loc[df['record_id_dem_endo'] == str(int(self.subjectId))]

        if len(record) is 0 or None:
            QMessageBox.about(self, "WARNING", "--- Please create subject %s form first" % str(self.subjectId)  )
            return
            
        studyType_redcap = record.get('redcap_event_name').values
        
        #"CD_ENDO", "CTL_ENDO", "CD_Surgery","CTL_Surgery"
        if studyType_redcap == 'cd_arm_1':
            studyType_checker = 'CD_ENDO'
        elif studyType_redcap == 'control_arm_1':
            studyType_checker = 'CTL_ENDO'
        elif studyType_redcap == 'cd_arm_2':
            studyType_checker = 'CD_Surgery'
        elif studyType_redcap == 'control_arm_2':
            studyType_checker = 'CTL_Surgery'
            
        if studyType_checker != self.studyType:
            print(studyType_redcap)
            QMessageBox.about(self, "WARNING", "Study type for subject %s does not match. Should it be %s?" % (str(self.subjectId),str(studyType_checker)))
            return
        
#         if self.custom_option =='' or self.custom_option == None:
#             QMessageBox.about(self, "WARNING", "Custom option need to set and\nit should only be:\n Need extra FROZEN specimens." )
#             return
        
#         if self.custom_option =='Default' or self.custom_option == 'No FRESH specimens, NO extra FROZEN specimens':
#             QMessageBox.about(self, "WARNING", "Custom option need to set and\nit should only be:\nNeed extra FROZEN specimens." )
#             return
        
        ###add constraint to restrict one patient can only be one type
        
        self.custom_option = self.redcapExec.getIfFreshNeedOrNot(self.subjectId)
        if self.custom_option =='' or self.custom_option == None:
            QMessageBox.about(self, "WARNING", "Subject %s id not found. Print new pack first" % str(self.subjectId) )
            return

        print(self.custom_option)
        
        # all set, ready to generate barcode pack
        barcodeExec = BarcodeGenerator(self.subjectId,self.studyType)        
        #barcodeExec.execute()
        
        
        #can optimize - done?
        nextAvailActionId = self.redcapExec.getNextAvailActionId()
        
        # new feature 12/19/2019
        #tmp_barcode_list = barcodeExec.get_barcode_list(self.custom_option) # More constraint before Christmas...
        
        tmp_barcode_list = barcodeExec.get_extra_frozen_barcode_list() # a hack way to get ADD frozen id only   
        
        if len(tmp_barcode_list) is not 4: # only 4 extra barcode ids. 
            QMessageBox.about(self, "WARNING", "Something wrong, please contact adminstrater")
            return
        
        #set generate action
        for tmp_barcode_id in tmp_barcode_list:
            if tmp_barcode_id == '': # no need to print
                continue 
                
            ean8_code = barcodeExec.recordId_to_ean8(tmp_barcode_id)
            
            # generate ean8 code to make sure it matches our criteria
            if ean8_code is None:
                QMessageBox.about(self, "WARNING", "Something wrong with barcode id %s?" % (tmp_barcode_id))
                return
            
            self.printLabel(tmp_barcode_id, ean8_code)
            # add barcode ean8 code
            self.redcapExec.setPrintedAction(nextAvailActionId, tmp_barcode_id, self.studyType,self.processed_by,self.action_comment,ean8_code,self.custom_option)
            nextAvailActionId += 1
        
            bool_update_redcap_sophie = self.redcapExec.update_REDCAP_SOPHIE_EXTRA_FROZEN(self.studyType,str(int(self.subjectId)),tmp_barcode_list, self.custom_option)
        
        self.plain_comment.clear()
        
        curTime = datetime.now()
        cur_date = curTime.strftime('%Y-%m-%d %H:%M')
        self.plain_console.appendPlainText("\n\n%s\nGenerate Extra Frozen Barcodes IDs for subject %s DONE! Printing may takes time, please be patient.\nTotal barcodes to print: %s" 
                                           % (str(cur_date),str(self.subjectId),str(len(tmp_barcode_list))))
        
        self.cb_option_type.setCurrentIndex(0)
        self.plain_console.repaint()
    
    @pyqtSlot()
    def printExtraSerumOnly(self):
        """
        Generate and print new pack of barcodes
        """
        self.getGUIFieldValue()
        if self.subjectId == '' or self.subjectId == None:
            QMessageBox.about(self, "WARNING", "Subject id is empty. Print extra frozen specimen for subject %s failed." % str(self.subjectId)  )
            return
        
        if len(self.subjectId) != 3:
            QMessageBox.about(self, "WARNING", "Subject id should be three digit. Please double-check" )
            return
        
        if self.studyType == '' or self.studyType == None:
            QMessageBox.about(self, "WARNING", "Study type is empty. Generate / Print new pack for subject %s failed." % str(self.subjectId)  )
            returnge
        
        if self.processed_by == '' or self.processed_by ==None:
            QMessageBox.about(self, "WARNING", "Please select current user." )
            return
   
        curSubj = self.redcapExec.getInputPatient()
        df = pd.DataFrame.from_dict(curSubj)
        record = df.loc[df['record_id_dem_endo'] == str(int(self.subjectId))]

        if len(record) is 0 or None:
            QMessageBox.about(self, "WARNING", "--- Please create subject %s form first" % str(self.subjectId)  )
            return
            
        studyType_redcap = record.get('redcap_event_name').values
        
        #"CD_ENDO", "CTL_ENDO", "CD_Surgery","CTL_Surgery"
        if studyType_redcap == 'cd_arm_1':
            studyType_checker = 'CD_ENDO'
        elif studyType_redcap == 'control_arm_1':
            studyType_checker = 'CTL_ENDO'
        elif studyType_redcap == 'cd_arm_2':
            studyType_checker = 'CD_Surgery'
        elif studyType_redcap == 'control_arm_2':
            studyType_checker = 'CTL_Surgery'
            
        if studyType_checker != self.studyType:
            print(studyType_redcap)
            QMessageBox.about(self, "WARNING", "Study type for subject %s does not match. Should it be %s?" % (str(self.subjectId),str(studyType_checker)))
            return
    
        self.custom_option =  self.redcapExec.delGetSubjectLastestCustomOption(self.subjectId) # hacking  since this function is gonna removed..
        if self.custom_option =='' or self.custom_option == None:
            QMessageBox.about(self, "WARNING", "Subject %s id not found. Print new pack first" % str(self.subjectId) )
            return

        print(self.custom_option)
        
        # all set, ready to generate barcode pack
        barcodeExec = BarcodeGenerator(self.subjectId,self.studyType)
        
        #can optimize - done?
        nextAvailActionId = self.redcapExec.getNextAvailActionId()

        tmp_barcode_list = barcodeExec.get_extra_serum_barcode_list() # a hack way to get ADD frozen id only   
        
        if len(tmp_barcode_list) is not 3: # only 4 extra barcode ids. 
            QMessageBox.about(self, "WARNING", "Something wrong, please contact adminstrater")
            return
        
        #set generate action
        for tmp_barcode_id in tmp_barcode_list:
            if tmp_barcode_id == '': # no need to print
                continue 
                
            ean8_code = barcodeExec.recordId_to_ean8(tmp_barcode_id)
            
            # generate ean8 code to make sure it matches our criteria
            if ean8_code is None:
                QMessageBox.about(self, "WARNING", "Something wrong with barcode id %s?" % (tmp_barcode_id))
                return
            
            self.printLabel(tmp_barcode_id, ean8_code)
            # add barcode ean8 code
            self.redcapExec.setPrintedAction(nextAvailActionId, tmp_barcode_id, self.studyType,self.processed_by,self.action_comment,ean8_code,self.custom_option)
            nextAvailActionId += 1
        
            bool_update_redcap_sophie = self.redcapExec.update_REDCAP_SOPHIE_EXTRA_SERUM(self.studyType,str(int(self.subjectId)),tmp_barcode_list, self.custom_option)
        
        self.plain_comment.clear()
        
        curTime = datetime.now()
        cur_date = curTime.strftime('%Y-%m-%d %H:%M')
        self.plain_console.appendPlainText("\n\n%s\nGenerate new extra Serum barcodes ids for subject %s DONE! Printing may takes time, please be patient.\nTotal barcodes to print: %s" 
                                           % (str(cur_date),str(self.subjectId),str(len(tmp_barcode_list))))
        
        self.cb_option_type.setCurrentIndex(0)
        self.plain_console.repaint()
        
    @pyqtSlot()
    def genPrintNewPack(self):
        """
        Generate and print new pack of barcodes
        """
        self.getGUIFieldValue()
        if self.subjectId == '' or self.subjectId == None:
            QMessageBox.about(self, "WARNING", "Subject id is empty. Generate / Print new pack for subject %s failed." % str(self.subjectId)  )
            return
        
        if len(self.subjectId) != 3:
            QMessageBox.about(self, "WARNING", "Subject id should be three digit. Please double-check" )
            return
        
        if self.studyType == '' or self.studyType == None:
            QMessageBox.about(self, "WARNING", "Study type is empty. Generate / Print new pack for subject %s failed." % str(self.subjectId)  )
            return
        
        if self.processed_by == '' or self.processed_by ==None:
            QMessageBox.about(self, "WARNING", "Please select current user." )
            return
        
        if self.custom_option =='' or self.custom_option == None:
            QMessageBox.about(self, "WARNING", "Please select custom option for printing." )
            return
        
        ###add constraint to restrict one patient can only be one type
        curSubj = self.redcapExec.getInputPatient()
        df = pd.DataFrame.from_dict(curSubj)
        record = df.loc[df['record_id_dem_endo'] == str(int(self.subjectId))]

        if len(record) is 0 or None:
            QMessageBox.about(self, "WARNING", "--- Please create subject %s form first" % str(self.subjectId)  )
            return
            
        studyType_redcap = record.get('redcap_event_name').values
        
        #"CD_ENDO", "CTL_ENDO", "CD_Surgery","CTL_Surgery"
        if studyType_redcap == 'cd_arm_1':
            studyType_checker = 'CD_ENDO'
        elif studyType_redcap == 'control_arm_1':
            studyType_checker = 'CTL_ENDO'
        elif studyType_redcap == 'cd_arm_2':
            studyType_checker = 'CD_Surgery'
        elif studyType_redcap == 'control_arm_2':
            studyType_checker = 'CTL_Surgery'
            
        if studyType_checker != self.studyType:
            print(studyType_redcap)
            QMessageBox.about(self, "WARNING", "Study type for subject %s does not match. Should it be %s?" % (str(self.subjectId),str(studyType_checker)))
            return
        
        # all set, ready to generate barcode pack
        barcodeExec = BarcodeGenerator(self.subjectId,self.studyType)
        
        barcodeExec.execute()
        
        #can optimize - done?
        nextAvailActionId = self.redcapExec.getNextAvailActionId()
        
        # new feature 12/19/2019
        tmp_barcode_list = barcodeExec.get_barcode_list(self.custom_option) # More constraint before Christmas...
           
        
        #set generate action
        for tmp_barcode_id in tmp_barcode_list:
            if tmp_barcode_id == '': # no need to print
                continue 
                
            ean8_code = barcodeExec.recordId_to_ean8(tmp_barcode_id)
            
            # generate ean8 code to make sure it matches our criteria
            if ean8_code is None:
                QMessageBox.about(self, "WARNING", "Something wrong with barcode id %s?" % (tmp_barcode_id))
                return
            
            self.printLabel(tmp_barcode_id, ean8_code)
            # add barcode ean8 code
            self.redcapExec.setPrintedAction(nextAvailActionId, tmp_barcode_id, self.studyType,self.processed_by,self.action_comment,ean8_code,self.custom_option)
            nextAvailActionId += 1
        
            bool_update_redcap_sophie = self.redcapExec.update_REDCAP_SOPHIE(self.studyType,str(int(self.subjectId)),tmp_barcode_list, self.custom_option)
        
        self.plain_comment.clear()
        
        curTime = datetime.now()
        cur_date = curTime.strftime('%Y-%m-%d %H:%M')
        self.plain_console.appendPlainText("\n\n%s\nGenerate new pack for subject %s DONE!\nCustom option:%s\n Printing may takes time, please be patient.\nTotal barcodes to print: %s" 
                                           % (str(cur_date),str(self.subjectId),str(self.custom_option),str(len(tmp_barcode_list))))
        
        self.cb_option_type.setCurrentIndex(0)
        self.plain_console.repaint()
        

    @pyqtSlot()
    def rePrintSingleLabel(self, tmp_ean8_code):
        """
        Reprint label
        """
        self.printLabel(self.barcode_id,tmp_ean8_code)
        curTime = datetime.now()
        cur_date = curTime.strftime('%Y-%m-%d %H:%M')
#        self.plain_console.appendPlainText("\n\n%s\nReprint barcode %s DONE!" % (str(cur_date),str(self.barcode_id)))
    
    def printLabel(self, _barcode_id,_ean8_code):
        """
        Use dymo printer to print label 
        """
        tmpLabelPrint = LabelPrintHelper_mac(_barcode_id, _ean8_code)
        tmpLabelPrint.execute()
        return True 
    
    @pyqtSlot()
    def setStatusSetAction(self):
        """
        Set all actions of individual barcode
        """
        self.getGUIFieldValue()
        
 #       if self.barcode_id is '' or self.barcode_id is None:
#         if len(self.barcode_id) != 8:
#             QMessageBox.about(self, "WARNING", "Barcode id is invalid. Action failed. \n\n Please re-scan  barcode id box" )
#             return
        
        if self.studyType == '' or self.studyType == None:
            QMessageBox.about(self, "WARNING", "Study type is empty. Do action for barcode %s failed." % str(self.barcode_id))
            return
        
        if self.processed_by == '' or self.processed_by ==None:
            QMessageBox.about(self, "WARNING", "Please select current user." )
            return
        
        # check barcode exists and return custom option
        self.subjectId = self.barcode_id[3:6] #GCAxxx -> xxx
        tmp_custom_option = self.redcapExec.getSubjectLastestCustomOption(self.barcode_id,self.subjectId)
        if tmp_custom_option is False:
            QMessageBox.about(self, "WARNING", 
                              "Barcode %s is not found \n\n please generate barcode first" % str(self.barcode_id))
            return
#         barcode_checker = self.redcapExec.ifBarcodeExist(self.barcode_id)
#         if barcode_checker is False:
#             QMessageBox.about(self, "WARNING", 
#                               "Barcode %s is not found \n\n please generate barcode first" % str(self.barcode_id))
#             return

        
    
        nextAvailActionId = self.redcapExec.getNextAvailActionId()
        self.subjectId = self.barcode_id[3:6]
        # add real barcode to ean8 code
        barcodeExec = BarcodeGenerator(self.subjectId,self.studyType)
        ean8_code = barcodeExec.recordId_to_ean8(self.barcode_id)

        if ean8_code is None:
            QMessageBox.about(self, "WARNING", "Something wrong with barcode id %s? \n Or please double-check your subject type" % (self.barcode_id))
            return
        
        # re-print has unique operation
        if self.action_type == 'Re-print':
            self.rePrintSingleLabel(ean8_code)
            
        self.redcapExec.setAction(nextAvailActionId, self.barcode_id, self.studyType,self.processed_by,self.action_type,self.action_comment,ean8_code,tmp_custom_option)

        self.plain_comment.clear()
        curTime = datetime.now()
        cur_date = curTime.strftime('%Y-%m-%d %H:%M')
        self.plain_console.appendPlainText("\n\n%s\nbarcode %s action: %s DONE!" % (str(cur_date),str(self.barcode_id),self.action_type))
        self.plain_console.repaint()
        self.textbox_barcode_id.selectAll()
        
    @pyqtSlot(QImage)
    def set_webcam_image(self, image):
        """
        Get video image from webcam and print to GUI
        """
        self.cam_label.setPixmap(QPixmap.fromImage(image))

    @pyqtSlot()
    def getBarcodeFromScanner(self):
        """
        Read barcode ean8 code from webcam
        """
        self.getGUIFieldValue()
        if self.processed_by == '' or self.processed_by ==None:
            QMessageBox.about(self, "WARNING", "Please select current user." )
            return
        
        self.set_scanned_id(self.textbox_barcode_id.text())

    @pyqtSlot('QString')
    def set_scanned_id(self, scanned_id):
        """
        Convert ean8 id to real barcode id
        """
#         scanned_id = '30500020'
        barcodeExec = BarcodeGenerator()
    
        # hacking in case the input code is GCA blabla blabla      
        if scanned_id[0:3] == 'GCA':
            self.barcode_id = scanned_id
            scanned_id = barcodeExec.recordId_to_ean8(self.barcode_id)
        
        if barcodeExec.is_valid(str(scanned_id)) is False:
            QMessageBox.about(self, "WARNING", "Barcode is not valid(1). Please contact to administrator" )
            self.plain_console.repaint()
            return
        
        tmp_barcode_id = barcodeExec.ean8_to_recordId(scanned_id)
        if tmp_barcode_id is None:
            QMessageBox.about(self, "WARNING", "Barcode is not valid(2). Please contact to administrator" )
            self.plain_console.repaint()
            return
    
#         barcode_checker = self.redcapExec.ifBarcodeExist(tmp_barcode_id)
#         if barcode_checker is False:
#             QMessageBox.about(self, "WARNING", 
#                               "Barcode %s is not found \n\n please generate barcode first" % str(self.barcode_id))
#             return
        # check barcode exists and return custom option

        
        
        # set GUI
        self.barcode_id = tmp_barcode_id
        self.textbox_barcode_id.setText(self.barcode_id)
        self.subjectId = self.barcode_id[3:6]
        tmp_custom_option = self.redcapExec.getSubjectLastestCustomOption(self.barcode_id,self.subjectId)
        if tmp_custom_option is False:
            QMessageBox.about(self, "WARNING", 
                              "Barcode %s is not found \n\n please generate barcode first" % str(self.barcode_id))
            return
        
        tmp_studyType = scanned_id[0:1]
        
        if tmp_studyType == '1':
            self.cb_study_type.setCurrentIndex(1)
        elif tmp_studyType == '2':
            self.cb_study_type.setCurrentIndex(2)
        elif tmp_studyType == '3':
            self.cb_study_type.setCurrentIndex(3)
        elif tmp_studyType == '4':
            self.cb_study_type.setCurrentIndex(4)

        nextAvailActionId = self.redcapExec.getNextAvailActionId()
        self.redcapExec.setScannedAction(nextAvailActionId, self.barcode_id, self.studyType,self.processed_by,self.action_comment,scanned_id,tmp_custom_option)
        self.plain_comment.clear()
        curTime = datetime.now()
        cur_date = curTime.strftime('%Y-%m-%d %H:%M')
        self.plain_console.appendPlainText("\n\n%s\nSet barcode %s scanned status DONE!" % (str(cur_date),str(self.barcode_id)))
        self.plain_console.repaint()
                
    @pyqtSlot()
    def setStatusScanned(self):
        """
        Start a webcam thread
        """
        self.getGUIFieldValue()
        self.th_webcam.start()

        
    @pyqtSlot()   
    def getBarcodeStats(self):
        """
        Simple project / subject / barcode stats
        """
        self.getGUIFieldValue()
        
        if self.stats_type == 'Project':
            proj_stats = self.redcapExec.getProjectStats()
            curTime = datetime.now()
            cur_date = curTime.strftime('%Y-%m-%d %H:%M')
            self.plain_console.appendPlainText("\n\n%s\nProject Stats:\n%s" % (str(cur_date),proj_stats))
            
        elif self.stats_type == 'Subject':
            
            if self.subjectId == '' or self.subjectId == None:
                QMessageBox.about(self, "WARNING", "Subject id is empty. Get stats for subject %s failed." % str(self.subjectId)  )
                return
        
            self.getSubjectType()  # --> force to select correct subject type
            
            if self.studyType == '' or self.studyType == None:
                QMessageBox.about(self, "WARNING", "Study type is empty. Get stats for subject %s failed." % str(self.subjectId))
                return
            
            # a hacking here because we don't care if the subject contains extra frozen id or not.
            barcodeExec = BarcodeGenerator(self.subjectId,self.studyType)
            
            barcodeExec.execute()
            tmp_barcode_list = barcodeExec.get_barcode_list(self.custom_option)  
            
            subj_stats = self.redcapExec.getSubjectStats(tmp_barcode_list)
            
            curTime = datetime.now()
            cur_date = curTime.strftime('%Y-%m-%d %H:%M')
            self.plain_console.appendPlainText("\n\n%s\nSubject %s Stats:\n%s" % (str(cur_date),self.subjectId, subj_stats))
         
        elif self.stats_type == 'Barcode':
            if self.barcode_id is '' or self.barcode_id is None:
                QMessageBox.about(self, "WARNING", "Barcode id is empty. Set scan status failed. \n\n Please fill up barcode id box" )
                return
            barcode_stats = self.redcapExec.getBarcodeStats(self.barcode_id)
            
            curTime = datetime.now()
            cur_date = curTime.strftime('%Y-%m-%d %H:%M')
            self.plain_console.appendPlainText("\n\n%s\nBarcode %s Stats is stored in :\n%s" % (str(cur_date),self.barcode_id, barcode_stats))

        self.plain_console.repaint()
        
            
    def getPatentId(self,cur_id):
        ""
        int_cur_id = int(cur_id)
        if int_cur_id < 10:
            return '00%s' % str(int_cur_id)
        elif int_cur_id < 100:
            return '0%s' % str(int_cur_id)
        else:
            return '%s' % str(int_cur_id)
        

if __name__ == '__main__':
    # create tmp directory for barcode label generation

    
    app = QApplication(sys.argv)
    ex = RedcapLabelwriterApp_v3()
    sys.exit(app.exec_())
    
    # clean up barcode label tmp directory
    curDir = os.getcwd()
    barcode_tmp_path = curDir + '/tmp'
    
    if os.path.exists(barcode_tmp_path):
        shutil.rmtree(barcode_tmp_path)
    



Creation of the directory /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp failed
inCD_ENDO
outCD_ENDO
14
1
7
GCA901ST1
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901ST1_label.png
15
1
4
GCA901ST2
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901ST2_label.png
16
1
1
GCA901ST3
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901ST3_label.png
17
1
8
GCA901ST4
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901ST4_label.png
18
1
5
GCA901SR1
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webca

17
2
7
GCA901SR10
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901SR10_label.png
18
2
4
GCA901SR11
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901SR11_label.png
13
1
0
GCA901DNA
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901DNA_label.png
11
0
7
GCA901FreshTIA
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901FreshTIA_label.png
12
0
4
GCA901FixedTIA
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA901FixedTIA_label.png
13
0
1
GCA901FrozenTIA
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRe

outCD_Surgery
15
0
5
GCA903FreshTI
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA903FreshTI_label.png
16
0
2
GCA903FixedTI
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA903FixedTI_label.png
17
0
9
GCA903FrozenTI
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA903FrozenTI_label.png
18
0
6
GCA903FreshAC
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA903FreshAC_label.png
19
0
3
GCA903FixedAC
lpr -o PageSize=Custom.23x30mm   /Users/obo/Documents/My Research/LabelRedcapIntegration/DymoRedcapIntegration/v3_webcam_checksum_ean8/tmp/GCA903FixedAC_label.png
20
0
0
GCA903FrozenAC
lpr -o PageSize=Custom.23x30mm   /Users/ob

SystemExit: 0

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import os
import shutil
curDir = os.getcwd()
if not os.path.exists(curDir +'/tmp'):
    os.mkdir(curDir +'/tmp')



In [1]:
ls

Arialn.ttf
GCA.png
REDCAP_API_KEY.txt
RedcapLabelwriterApp_v3.ipynb
__pycache__/
barcodeGenerator.py
barcode_reader.py
bk_redcapLabelwriterApp_v3_mac.py
cheatSheetEAN_Converter.py
exceptions.py
labelPrintHelper_mac.py
masi.png
redcapHelper.py
redcapLabelwriterApp_v3_mac.py
scanner-beep.wav
tmp/
webcam.png
with_comment_RedcapLabelwriterApp_v3.ipynb


In [1]:
pip install pynput

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install playsound

SyntaxError: invalid syntax (<ipython-input-4-e50db89bef97>, line 2)

In [1]:
pip install pyobjc

  Using cached https://files.pythonhosted.org/packages/b1/c9/83b256bb7f771a9e4232518005d6f237ae79215283cf5ea234141751b36a/pyobjc-6.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/82/6e/ef84fc0483e068b41d97ef2e49fb56c2469b67a3aaedde9106f227de092e/pyobjc_framework_CoreWLAN-6.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/05/65/bb454732f8d741cdfdfb3fb5b554b134c405d968e9717da342c21edaf039/pyobjc_framework_PubSub-6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/84/22/217502fa0edfce99f53e83a4e71a8ddacaed9aad3015e897dd5b0d504199/pyobjc_framework_NotificationCenter-6.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/40/29/e5f6039b658e692a4275fdcb4c416ae1de6e11fca88a104d5e8deaf84ae4/pyobjc_framework_GameplayKit-6.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/bc/cc/6349c0ce531d11eca5e92562f52550c0ac7f6e018d80fd6

  Using cached https://files.pythonhosted.org/packages/cb/55/184ff7b391880d996e4a87fd6b3ecbefc6c55753c0c99f01126581a86228/pyobjc_framework_SpriteKit-6.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/2d/8d/ef6367d06318177caf9e8d4868684d18cd0cc65efba25263332c2b7ae133/pyobjc_framework_OpenDirectory-6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9f/05/9fb0c122de977ddaa99fbab1b1bf9d0eff291eb70be7e2430ef9002085d4/pyobjc_framework_CoreServices-6.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e4/f5/421abd30eb21f6457caabb609f76127b283c7f7a93fce423c3f9825c97fa/pyobjc_framework_CalendarStore-6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/82/ef1db1f42b2a2d0ee261b3fc6195c7177feefc8cf1d55bb486b188c003fc/pyobjc_framework_WebKit-6.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/62/3d/bdf55ccfc8253358e97f3d

  Using cached https://files.pythonhosted.org/packages/a4/e7/3e3b3ac525dd04e8a810801f5a2b66c819043927cef6bdc79b7e1747dea5/pyobjc_framework_iTunesLibrary-6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/96/b6/97253043d0850de8f2d4b73ecd4fb47a2b7e4d2875c21fad876b28898be3/pyobjc_framework_ServiceManagement-6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5a/06/6b462c9123b865679008935c874636aef87298072994054c074dde81d62e/pyobjc_framework_VideoToolbox-6.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/db/f2/c0f3908574080cfce42d90f2168019f0f7b6299415edebf9ad926607b5d5/pyobjc_framework_DiscRecording-6.1-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/95/7b/b51ef3350ba7915c36f354b401dde9fd82446bab6127e3a2900ce302ff02/pyobjc_framework_Social-6.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0f/a2/7a4d7a487caabbf7678c19ed20a

Note: you may need to restart the kernel to use updated packages.


In [1]:
from redcapHelper import RedcapHelper
import redcap
import pandas as pd
import numpy as np

def getPatentId(cur_id):
    ""
    int_cur_id = int(cur_id)
    if int_cur_id < 10:
        return '00%s' % str(int_cur_id)
    elif int_cur_id < 100:
        return '0%s' % str(int_cur_id)
    else:
        return '%s' % str(int_cur_id)
        

proj = redcap.Project('https://redcap.vanderbilt.edu/api/', '32F2DE4F8CD17005938DD49009A45468')

# printSubj = proj.export_records(events=['cd_arm_1','control_arm_1','cd_arm_2','control_arm_2','action_tuple_table_arm_3'],fields=['record_id_dem_endo','barcode_sample_id'])

# df = pd.DataFrame.from_dict(printSubj)
#print(df)


printSubj = proj.export_records(events=['action_tuple_table_arm_3'],fields=['barcode_sample_id'])

df = pd.DataFrame.from_dict(printSubj)
barcodeList  = df['barcode_sample_id'].values


real_barcodeList = []
for x in barcodeList:
    real_barcodeList.append(int(x[3:6]))
    
real_barcodeList_np = np.asarray(real_barcodeList)
real_barcodeList_np_unique = np.unique(real_barcodeList_np)

next_print_subject_id = ''
if len(real_barcodeList_np_unique) is 0:
    next_print_subject_id = 1
else:
    tmp = 0
    for i in real_barcodeList_np_unique:
        if i - tmp == 1:
            tmp = i
        else:
            next_print_subject_id = tmp + 1
            
# print(next_print_subject_id)
# print(big_array_np_unique)

curSubj = proj.export_records(events=['cd_arm_1','control_arm_1','cd_arm_2','control_arm_2'],fields=['record_id_dem_endo'])
df = pd.DataFrame.from_dict(curSubj)
record = df.loc[df['record_id_dem_endo'] == str(next_print_subject_id)]

if len(record) is 0 or None:
    print('Please create subject %s first' % str(next_print_subject_id))

type_of_subject = record.get('redcap_event_name').values
next_print_subject_id_verified = getPatentId(next_print_subject_id)


print(record.get('redcap_event_name').values)


# df = pd.DataFrame.from_dict(printSubj)
# for i in big_array_np:

# print(barcodeList[len(barcodeList)-1])

# testList = df['record_id_dem_endo','redcap_event_name'].values
# print(testList[len(testList)-1])

# print(printSubj[len(printSubj)-1].get('record_id_dem_endo'))
#data = pd.DataFrame.from_dict(tmp_subset)


#subset = proj.export_records(events=['cd_arm_1','control_arm_1','cd_arm_2','control_arm_2'],fields=['record_id_dem_endo'])
# print(subset[len(subset) - 1].get('record_id_dem_endo'))
# print(subset[len(subset) - 1].get('redcap_event_name'))
# print(subset[len(subset) - 1])
# subset = proj.export_records()


['cd_arm_1']


In [5]:
from cheatSheetEAN_Converter import CheatSheetEAN_Converter
t= CheatSheetEAN_Converter()
print(len(t.CD_ENDO_CHEAT_LIST))
print(len(t.CTL_ENDO_CHEAT_LIST))
print(len(t.CD_Surgery_CHEAT_LIST))
print(len(t.CTL_Surgery_CHEAT_LIST))
print(t.CTL_Surgery_CHEAT_LIST.index('FreshTI1k'))

25
19
6
24


ValueError: 'FreshTI1k' is not in list

In [2]:
a = 'GCA001Frozenabcde'
print(a[1:3])

CA


In [4]:
from barcodeGenerator import BarcodeGenerator

b = BarcodeGenerator('001','CD_ENDO')
x = b.recordId_to_ean8('GCA001SR8')
if x is None:
    print('blablabala')
print(x[4:7])

ahahahahha1240001
6
2
0
001


In [4]:
import os
print (os.chdir(os.getcwd()))

None


In [2]:
from redcapHelper import RedcapHelper
import pandas as pd
redcapExec = RedcapHelper('32F2DE4F8CD17005938DD49009A45468')

data = redcapExec.project.export_records()
print(data[151])

# #print(type(data[14]))
# #print(type(data[15]))
# dd = {k: v for k, v in data[17].items() if v is not ''}

# print(len(dd))

# for i in dd:
#     print('%s:%s' % (i,dd.get(i)))

{'record_id_dem_endo': 'action135', 'redcap_event_name': 'action_tuple_table_arm_3', 'consent_date_dem_endo': '', 'endoscopy_date_dem_endo': '', 'mrn_dem_endo': '', 'first_name_dem_endo': '', 'last_name_dem_endo': '', 'dob_dem_endo': '', 'age_dem_endo': '', 'gender_dem_endo': '', 'dem_endo_race': '', 'height_dem_endo': '', 'weight_dem_endo': '', 'consent_date_dem_sur': '', 'resection_date_dem_sur': '', 'mrn_dem_sur': '', 'first_name_dem_sur': '', 'last_name_dem_sur': '', 'dob_dem_sur': '', 'age_dem_sur': '', 'gender_dem_sur': '', 'dem_sur_race': '', 'height_dem_sur': '', 'weight_dem_sur': '', 'participant_name_econsent': '', 'participant_age_econsent': '', 'proceed_hipaa': '', 'future_studies': '', 'agree': '', 'patient_name': '', 'date_consented': '', 'gene_1': '', 'gene_2': '', 'gene_3': '', 'gene_sign': '', 'gene_date': '', 'consenter_name': '', 'time_consented': '', 'study_date_consent_info': '', 'coordinator_name_consent_info': '', 'consent_note_consent_info': '', 'consented_date_

In [1]:
from redcapHelper import RedcapHelper
import pandas as pd
redcapExec = RedcapHelper('32F2DE4F8CD17005938DD49009A45468')

data = redcapExec.project.export_records(['action_tuple_table_arm_3'],['barcode_sample_id'])
data


[]

In [10]:
from datetime import datetime

curTime = datetime.now()
print(curTime)
print(curTime.strftime('%Y-%m-%d %H:%M'))

2019-12-11 18:07:39.343691
2019-12-11 18:07


In [20]:
from redcapHelper import RedcapHelper
import pandas as pd
redcapExec = RedcapHelper('32F2DE4F8CD17005938DD49009A45468')

tmp_subset = redcapExec.project.export_records(events=['action_tuple_table_arm_3'],fields=['barcode_sample_id','barcode_action_type','barcode_processed_by'])

barcodeSubset = redcapExec.getBarcodeSubset()
if len(barcodeSubset) == 0: 
    print('No subject got printed yet!')
    # return

df = pd.DataFrame.from_dict(tmp_subset)
record = df.loc[df['barcode_action_type'] == 'printed']

if len(record) is 0 or None:
    print('Something wrong, please contact administrator!')
else:
    df_record = df.iloc[-1,:]
    act = df_record.tolist()
    #['action56', 'action_tuple_table_arm_3', 'GCA002FrozenACB', 'Sophie', 'printed']
    print(act[2][3:6])
    print(act[3])
    #record_list = record['barcode_sample_id''barcode_processed_by'].tolist()
    
    #print(record_list[len(record_list)-1][3:6])
    


    

002
Sophie


In [32]:
s = '2323 '
print(len(s))

5


In [27]:
from redcapHelper import RedcapHelper
import pandas as pd

# for i in range(1,57):
#     to_import = [{'redcap_event_name':'action_tuple_table_arm_3','record_id_dem_endo':'action%s'%str(i),'barcode_processed_by': 'Sophie'}]
# #to_import = [{'redcap_event_name':'cd_arm_1','record_id_dem_endo': '2'}]

# response = redcapExec.project.import_records(to_import)

###tmp_subset = redcapExec.project.export_records(events=['action_tuple_table_arm_3'],fields=['barcode_sample_id','barcode_action_type','barcode_custom_option'])

def getSubjectCustomOption(barcode_id,subject_id):
    """
    sdsd
    """  
    redcapExec = RedcapHelper('32F2DE4F8CD17005938DD49009A45468')
    tmp_subset = redcapExec.project.export_records(events=['action_tuple_table_arm_3'],fields=['barcode_sample_id','barcode_custom_option'])

    if len(tmp_subset) == 0:
        return False
        
    data = pd.DataFrame.from_dict(tmp_subset)
    df = data.loc[data['barcode_sample_id'] == barcode_id]
    if len(df) == 0:
        return False
    else:
            # result in 'action_tuple_table_arm_3','barcode_sample_id','barcode_custom_option'
        idx = len(tmp_subset)-1
        tmp_custom_option = ''
        while idx >= 0:
            print(tmp_subset[idx].get('barcode_sample_id'))
            tmp_subj = tmp_subset[idx].get('barcode_sample_id')[3:6]
            if tmp_subj == subject_id:
                tmp_custom_option = tmp_subset[idx].get('barcode_custom_option')
                break
            idx -= 1
        print(idx)  
        return tmp_custom_option
    
print(getSubjectCustomOption('GCA003ST1','003'))

GCA003ST1
106
Default


In [5]:
a = [{'a':'1','b':'2'}]
a[0].update({'c':'3','d':'4'})
a = []
print(a)

[]


In [9]:
idx = 10
while idx >= 0:
    print(idx)
    idx -= 1

10
9
8
7
6
5
4
3
2
1
0


In [3]:
from redcapHelper import RedcapHelper
import pandas as pd

redcapExec = RedcapHelper('32F2DE4F8CD17005938DD49009A45468')
for i in range(1,107):
    to_import = [{'redcap_event_name':'action_tuple_table_arm_3','record_id_dem_endo':'action%s'%str(i),'barcode_custom_option': 'Default'}]
#to_import = [{'redcap_event_name':'cd_arm_1','record_id_dem_endo': '2'}]

    response = redcapExec.project.import_records(to_import)

In [1]:
from redcapHelper import RedcapHelper

import pandas as pd
redcapExec =  RedcapHelper('32F2DE4F8CD17005938DD49009A45468')

data = redcapExec.project.getRecordFromRedcap(['cd_arm_1','control_arm_1','cd_arm_2','control_arm_2'],['record_id_dem_endo'])
# data = redcapExec.project.export_records(events=['action_tuple_table_arm_3'],['barcode_sample_id','barcode_custom_option'])

AttributeError: 'Project' object has no attribute 'getRecordFromRedcap'